In [1]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
import forcateri

In [3]:
import importlib
importlib.reload(forcateri)
importlib.reload(forcateri.model.dartsmodels.dartstcnmodel)
importlib.reload(forcateri.baltbestapi.baltbestaggregatedapidata)

<module 'forcateri.baltbestapi.baltbestaggregatedapidata' from '/home/user/DFKI/forcateri/forcateri/baltbestapi/baltbestaggregatedapidata.py'>

In [4]:
model = TCNModel(
    input_chunk_length=7,
    output_chunk_length=5,  
    dropout=0.1,            
    kernel_size=3,          
    batch_size=32,          
    n_epochs=50,           
    random_state=42,
    num_filters=5,
    likelihood=QuantileRegression([0.1, 0.5, 0.9]),
    #pl_trainer_kwargs = trainer_kwargs    
)

In [5]:
baltbest = BaltBestAggregatedAPIData(name='test', url="baltbest_url", local_copy="/home/user/DFKI/forcateri/_data/showcase_data.csv")

In [6]:
baltbest._fetch_from_cache()

In [7]:
observed = ['temperature_1_max','temperature_2_max', 'temperature_room_avg']

In [8]:
baltbest._separate_ts(target='q_hca', known='temperature_outdoor_avg', observed=observed)

In [9]:
baltbest.known[486]

feature                          temperature_outdoor_avg
representation                                     value
offset time_stamp                                       
0 days 2021-01-01 00:00:00+00:00                     NaN
       2021-01-01 01:00:00+00:00                     NaN
       2021-01-01 02:00:00+00:00                     NaN
       2021-01-01 03:00:00+00:00                     NaN
       2021-01-01 04:00:00+00:00                     NaN
...                                                  ...
       2021-01-09 19:00:00+00:00                     NaN
       2021-01-09 20:00:00+00:00                     NaN
       2021-01-09 21:00:00+00:00                     NaN
       2021-01-09 22:00:00+00:00                     NaN
       2021-01-09 23:00:00+00:00                     NaN

[216 rows x 1 columns]

In [10]:
from forcateri.model.dartsmodels.dartsmodeladapter import DartsModelAdapter
mm = DartsModelAdapter(model=model,target=baltbest.target, known=baltbest.known, observed=baltbest.observed, forecast_horizon=5)